<a href="https://colab.research.google.com/github/AmbiTyga/Task-Stylumia/blob/Basic/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Kaggle Download
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c stumbleupon

Saving kaggle.json to kaggle.json
  0% 0.00/21.5k [00:00<?, ?B/s]
100% 21.5k/21.5k [00:00<00:00, 8.56MB/s]
 94% 147M/157M [00:00<00:00, 181MB/s]
100% 157M/157M [00:00<00:00, 170MB/s]
 62% 5.00M/8.02M [00:00<00:00, 44.1MB/s]
100% 8.02M/8.02M [00:00<00:00, 51.1MB/s]
  0% 0.00/3.45M [00:00<?, ?B/s]
100% 3.45M/3.45M [00:00<00:00, 114MB/s]


In [2]:
!unzip -q /content/raw_content.zip

In [3]:
!git clone https://github.com/misja/python-boilerpipe.git
%cd python-boilerpipe
!pip install -r requirements.txt -q
!python setup.py install -q
%cd ..

Cloning into 'python-boilerpipe'...
remote: Enumerating objects: 264, done.
remote: Total 264 (delta 0), reused 0 (delta 0), pack-reused 264
Receiving objects: 100% (264/264), 30.06 KiB | 3.34 MiB/s, done.
Resolving deltas: 100% (91/91), done.
/content/python-boilerpipe
     |████████████████████████████████| 460kB 6.0MB/s 
/usr/lib/python3.7/distutils/dist.py:274: UserWarning: Unknown distribution option: 'install_requires'
  warnings.warn(msg)
running install
running build
running build_py
creating build
creating build/lib
creating build/lib/boilerpipe
copying src/boilerpipe/__init__.py -> build/lib/boilerpipe
creating build/lib/boilerpipe/extract
copying src/boilerpipe/extract/__init__.py -> build/lib/boilerpipe/extract
creating build/lib/boilerpipe/data
creating build/lib/boilerpipe/data/boilerpipe-1.2.0
copying src/boilerpipe/data/boilerpipe-1.2.0/boilerpipe-1.2.0.jar -> build/lib/boilerpipe/data/boilerpipe-1.2.0
creating build/lib/boilerpipe/data/boilerpipe-1.2.0/lib
copying src/

In [4]:
import pandas as pd 
import numpy as np
import json
from bs4 import BeautifulSoup
import re
from boilerpipe.extract import Extractor

## Preprocessing Functions

In [5]:
def extract_domain(df_url):
    # extract domains
    domain = df_url.str.lower().apply(lambda x:x.split('/')[2])
    domain_parts = domain.str.split('.')

    return domain_parts.apply(lambda x:'.'.join(x[-2:]) if x[-2] not in ['com', 'co'] else '.'.join(x[-3:]))

In [95]:
def clean_string(s):
  s = re.sub(r'[\r\n\t]', ' ',s)
  s = re.sub(r"\s+", ' ', s)
  return s.strip()

In [6]:
def boil_soup(urlid):
    filename = '/content/raw_content/' + urlid

    with open(filename, 'rb') as f:
        html = f.read()

        for parser in ["lxml", "xml", "html5lib"]:
            soup = BeautifulSoup(html, parser)
            if soup.body:
                return soup

        return BeautifulSoup(html)

In [7]:
def get_boiled(id):
  with open('/content/raw_content/'+id,'rb') as f:
    h = f.read()
  
  try:
    extractor = Extractor(extractor='ArticleExtractor', html = h)
  except:
    extractor = Extractor(extractor='ArticleExtractor', html = h.decode('iso-8859-1'))
  return str(extractor.getText())

In [84]:
item[letsee>1]['title'].values[0]

['InsidersHealth.com', 'Acne Vulgaris']

In [ ]:
import numpy as np
letsee = item['title'].apply(len)
for i in item[letsee>1]['title'].values:

  print(i[np.argmax([len(x.split()) for x in i])])

In [97]:
def get_title(id):
  
  TAGS_REMOVE = ['script', 'style']
  
  soup = boil_soup(id)


  for el in soup.find_all(TAGS_REMOVE):
    el.extract()
  
  texts = soup.find_all('title')
  texts = [clean_string(x.text) for x in texts]

  try:
    return texts[np.argmax([len(x.split()) for x in texts])]
  except ValueError:
    return np.nan

## Dataset

In [12]:
# !wget https://raw.githubusercontent.com/AmbiTyga/Task-Stylumia/main/TrainTest.7z
!7z x TrainTest.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 23870960 bytes (23 MiB)

Extracting archive: TrainTest.7z
--
Path = TrainTest.7z
Type = 7z
Physical Size = 23870960
Headers Size = 138
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      3% - Final_parsed.json                          7% - Final_parsed.json                         11% - Final_parsed.json                         15% - Final_parsed.json                         19% - Final_parsed.json                         24% - Final_parsed.json      

In [14]:
item = pd.read_json("/content/Final_parsed.json")

In [87]:
data = pd.concat([pd.read_csv('/content/train.tsv.zip',sep = '\t'),
                  pd.read_csv('/content/test.tsv.zip',sep = '\t')],ignore_index = True)

In [88]:
item = data.copy()

## Feature Extraction

In [89]:
item['real_url'] = item['url'].str.lower()

In [90]:
item['domain'] = extract_domain(item['real_url'])

In [91]:
item['tld'] = item['domain'].str.split('.').apply(lambda x:x[-1])

In [92]:
item['body'] = item['boilerplate'].apply(lambda x:json.loads(x)['body'])
item.rename(columns = {'url':'org_url'},inplace = True)

item.drop(columns = ['boilerplate'],inplace = True)

In [41]:
item['boilerpipe'] = item['urlid'].astype(str).apply(get_boiled)

In [98]:
item['title'] = item['urlid'].astype(str).apply(get_title)

In [45]:
item['boilerplate'] = item[['title','body']].values.tolist()

In [ ]:
item.to_json("Final_parsed.json")